In [8]:
import pandas as pd
import numpy as np

In [9]:
data = pd.read_excel('/home/shysu/code/solve-for-target/data/data.xlsx')

In [10]:
data.shape

(34660, 9)

In [11]:
data.head()

,product,source,categories,date,didPurchase,doRecommend,rating,reviews,title
0,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13T00:00:00.000Z,NaN,1.0,5.0,This product so far has not disappointed. My c...,brand name
1,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13T00:00:00.000Z,NaN,1.0,5.0,great for beginner or experienced person. Boug...,very fast
2,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13T00:00:00.000Z,NaN,1.0,5.0,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.
3,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-13T00:00:00.000Z,NaN,1.0,4.0,I've had my XYZ brand HD 8 two weeks now and I...,Good!!!
4,electronics brand product name Tablet A 10.1 T...,Target,"Electronics,iPad & Tablets,All Tablets,XYZ bra...",2021-01-12T00:00:00.000Z,NaN,1.0,5.0,I bought this for my grand daughter when she c...,Fantastic Tablet for kids


In [12]:
data.describe()

,didPurchase,doRecommend,rating
count,1.0,34066.000000,34627.000000
mean,1.0,0.959373,4.584573
std,NaN,0.197427,0.735653
min,1.0,0.000000,1.000000
25%,1.0,1.000000,4.000000
50%,1.0,1.000000,5.000000
75%,1.0,1.000000,5.000000
max,1.0,1.000000,5.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34660 entries, 0 to 34659
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product      27900 non-null  object 
 1   source       34660 non-null  object 
 2   categories   34660 non-null  object 
 3   date         34621 non-null  object 
 4   didPurchase  1 non-null      float64
 5   doRecommend  34066 non-null  float64
 6   rating       34627 non-null  float64
 7   reviews      34658 non-null  object 
 8   title        34654 non-null  object 
dtypes: float64(3), object(6)
memory usage: 2.4+ MB


In [14]:
data.columns

Index(['product', 'source', 'categories', 'date', 'didPurchase', 'doRecommend',
       'rating', 'reviews', 'title'],
      dtype='object')

# Preprocessing

In [15]:
data.drop(['didPurchase', 'date'], axis=1, inplace=True)

Column: Product

In [16]:
data['product'].isna().sum()

np.int64(6760)

In [17]:
data.dropna(subset=['product'], inplace=True)

In [18]:
data['product'].unique().size

60

Column: Source

In [19]:
data['source'].isna().sum()

np.int64(0)

In [20]:
data['source'].unique().size

1

In [21]:
data['source'].unique()

array(['Target'], dtype=object)

In [22]:
data.drop(['source'], axis=1, inplace=True)

Column: Category

In [23]:
data['categories'].dropna(inplace=True)

In [24]:
data['categories'].unique().size

32

Column: Rating

In [25]:
data['rating'].unique()

array([ 5.,  4.,  2.,  1.,  3., nan])

In [26]:
data['rating'].isna().sum()

np.int64(32)

In [27]:
# Get total and missing rating counts
total_counts = data['product'].value_counts()
missing_counts = data[data['rating'].isna()]['product'].value_counts()

# Join only on products with missing ratings (inner join)
summary = pd.DataFrame(missing_counts).join(
    total_counts, how='inner', lsuffix='_missing', rsuffix='_total'
)

# Rename columns for clarity
summary.columns = ['missing_ratings', 'total_reviews']

# Add percent missing
summary['percent_missing'] = (summary['missing_ratings'] / summary['total_reviews'] * 100).round(2)

# Optional: sort by percent_missing
summary = summary.sort_values('percent_missing', ascending=False)

# Reset index to make 'product' a regular column
summary = summary.reset_index()

# Now it will look like a clean table
print(summary.to_string(index=False))

                                                                                                           product  missing_ratings  total_reviews  percent_missing
                                                             brand name Keyboard,,,_x000D_\nbrand name Keyboard,,,                7              9            77.78
electronics brand product name Tablet A 10.1 Tablet, 8 HD Display, Wi-Fi, 32 GB - Includes Special Offers, Magenta               12            158             7.59
                                         XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Black                4            372             1.08
                                      Brand New electronics brand IPad16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,                5           1038             0.48
                                       XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta                4          10966             0.04


In [28]:
summary['product']

0    brand name Keyboard,,,_x000D_\nbrand name Keyb...
1    electronics brand product name Tablet A 10.1 T...
2    XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Inc...
3    Brand New electronics brand IPad16gb 7 Ips Dis...
4    XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Inc...
Name: product, dtype: object

In [29]:
for product in summary['product']:
    mask = data['product'] == product
    median_rating = data.loc[mask, 'rating'].median()
    data.loc[mask, 'rating'] = data.loc[mask, 'rating'].fillna(median_rating)

In [30]:
data['rating'].isna().sum()

np.int64(0)

Column: doRecommend

In [31]:
data['doRecommend'].isna().sum()

np.int64(491)

In [32]:
data['doRecommend'].unique()

array([ 1.,  0., nan])

In [33]:
data[data['doRecommend'].isna()].groupby('product')['product'].value_counts()

product
\nelectonics brand Home                                                                                                                                                                                                              9
All-New PocketBook Touch HD 3 E-reader - Black, 6 Glare-Free Touchscreen Display, Wi-Fi -  Includes Special Offers,,                                                                                                               212
Brand New electronics brand IPad16gb 7 Ips Display Tablet Wifi 16 Gb Blue,,,                                                                                                                                                         8
XYZ brand Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Green Kid-Proof Case                                                                                                                                                         6
XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, 

In [34]:
# Only fill doRecommend where it's NaN
data.loc[data['doRecommend'].isna(), 'doRecommend'] = np.where(
    data.loc[data['doRecommend'].isna(), 'rating'] >= 4, 1, 0
)


Column: Reviews

In [35]:
data[data['reviews'].isna()][['product', 'title', 'rating']]

,product,title,rating
3111,retail brand 5W USB Official OEM Charger and P...,Five Stars,5.0
6724,"XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Inc...",Value for Money,5.0


In [36]:
data[data['reviews'].isna()][['product', 'title', 'rating']]['product'].value_counts()

product
retail brand 5W USB Official OEM Charger and Power Adapter for XYZ brand Tablets and brand name eReaders,,,_x000D_\nretail brand 5W USB Official OEM Charger and Power Adapter for XYZ brand Tablets and brand name eReaders,,,    1
XYZ brand Tablet, 7 Display, Wi-Fi, 8 GB - Includes Special Offers, Magenta                                                                                                                                                        1
Name: count, dtype: int64

In [37]:
data.dropna(subset=['reviews'], inplace=True)

Column: Title

In [38]:
data['title'].isna().sum()

np.int64(4)

In [39]:
data['title'] = data['title'].fillna('(missing title)')

# End of Preprocessing - Lets See The Data

In [40]:
data.shape

(27898, 6)

In [41]:
data.describe()

,doRecommend,rating
count,27898.000000,27898.000000
mean,0.956377,4.565471
std,0.204259,0.742437
min,0.000000,1.000000
25%,1.000000,4.000000
50%,1.000000,5.000000
75%,1.000000,5.000000
max,1.000000,5.000000


Create column: Feedback = Title + Reviews

In [42]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27898 entries, 0 to 27899
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product      27898 non-null  object 
 1   categories   27898 non-null  object 
 2   doRecommend  27898 non-null  float64
 3   rating       27898 non-null  float64
 4   reviews      27898 non-null  object 
 5   title        27898 non-null  object 
dtypes: float64(2), object(4)
memory usage: 1.5+ MB


# Feature Engineering

Combine title and reviews column for sentiment analysis

In [43]:
data['feedback'] = data['title'].astype(str) + data['reviews']

Clean up the product column to avoid duplicates and variation of same product

In [44]:
import re

def clean_product_name(text):
    text = str(text).lower().strip()

    # Remove excessive commas, whitespace
    text = re.sub(r',+', '', text)
    text = re.sub(r'\s{2,}', ' ', text)

    # Remove noise terms
    noise_terms = [
        'brand name', 'xyz brand', 'retail brand', 'electronics brand',
        'certified refurbished', 'includes special offers',
        'with special offers', 'special offers', 'previous generation',
        'new', 'brand new', 'official oem', 'product name'
    ]
    for term in noise_terms:
        text = text.replace(term, '')

    # Remove extra punctuation and whitespace again
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation

    return text.strip()

In [45]:
data['product'] = data['product'].apply(clean_product_name)

Clean up the categories column to tag products in a normalized manner

In [46]:
data['categories'] = data['categories'].str.lower().str.strip()

In [47]:
import re

brand_patterns = r"(all.*?|walmart.*?|target.*?|mazon.co.uk|retail brand.*?|xyz.*?|brand name.*?|see more.*?)(,|$)"
data['categories_clean'] = data['categories'].str.replace(brand_patterns, '', regex=True)
data['categories_clean'] = data['categories_clean'].str.replace(r",+", ",", regex=True).str.strip(", ")


In [48]:
categories = set()
for category in data['categories_clean']:
    categories.update(category.split(','))

In [49]:
categories

{'',
 ' audio player accessories',
 ' garage & office',
 ' movies',
 ' music',
 ' tech toys',
 ' wi-fi  3g (unlocked...',
 'alarms & sensors',
 'android tablets',
 'audio',
 'audio docks & mini speakers',
 'book accessories',
 'books & magazines',
 'cases & bags',
 'categories',
 'clearance',
 'computer accessories',
 'computer components',
 'computers',
 'computers & laptops',
 'computers & tablets',
 'computers/tablets & networking',
 'consumer electronics',
 'covers',
 'e-readers',
 'e-readers & accessories',
 'ebook readers',
 'ebook readers & accessories',
 'electronics',
 'electronics features',
 'ereaders',
 'ereaders & accessories',
 'featured brands',
 'frys',
 'hard drives & storage',
 'holiday shop',
 'home',
 'home & furniture clearance',
 'home automation',
 'home improvement',
 'home safety & security',
 'home security',
 'ipad & tablets',
 'ipads tablets',
 'ipod',
 "kids' tablets",
 'kitchen & dining features',
 'office',
 'office electronics',
 'pocketbook touch hd 3 e

In [50]:
data['category_list'] = data['categories_clean'].str.split(',')
data['category_list']

0        [electronics, ipad & tablets, tablets, compute...
1        [electronics, ipad & tablets, tablets, compute...
2        [electronics, ipad & tablets, tablets, compute...
3        [electronics, ipad & tablets, tablets, compute...
4        [electronics, ipad & tablets, tablets, compute...
                               ...                        
27895    [stereos, remote controls, audio docks & mini ...
27896    [stereos, remote controls, audio docks & mini ...
27897    [stereos, remote controls, audio docks & mini ...
27898    [stereos, remote controls, audio docks & mini ...
27899    [stereos, remote controls, audio docks & mini ...
Name: category_list, Length: 27898, dtype: object

In [51]:
# Define mapping from noisy tags to normalized taxonomy terms
normalization_map = {
    'ipad & tablets': 'tablets',
    'ipads tablets': 'tablets',
    'all tablets': 'tablets',
    'xyz brand tablets': 'tablets',
    'android tablets': 'tablets',
    'windows tablets': 'tablets',
    'tablets & ebook readers': 'tablets',
    'tablets & ereaders': 'tablets',
    'kids\' tablets': 'tablets',

    'e-readers': 'ereaders',
    'ebook readers': 'ereaders',
    'ebook readers & accessories': 'ereaders',
    'e-readers & accessories': 'ereaders',
    'ereaders & accessories': 'ereaders',
    'brand name e-readers': 'ereaders',

    'computers/tablets & networking': 'computers & tablets',
    'computers & laptops': 'computers & tablets',

    'audio docks & mini speakers': 'audio',
    'speaker systems': 'audio',
    'portable audio & headphones': 'audio',
    'audio player accessories': 'audio',

    'smart home & connected living': 'smart home',
    'smart home & home automation devices': 'smart home',
    'home safety & security': 'smart home',
    'smart hubs & wireless routers': 'smart home',
    'voice-enabled smart assistants': 'smart home',
    'voice assistants': 'smart home',
    'virtual assistant speakers': 'smart home',
    'alarms & sensors': 'smart home',

    'cases & bags': 'accessories',
    'tablet accessories': 'accessories',
    'tablet cases covers': 'accessories',
    'power adapters': 'accessories',
    'power adapters & cables': 'accessories',
    'computer accessories': 'accessories',
    'covers': 'accessories',

    'brand name e-reader accessories': 'accessories',
    'brand name paperwhite accessories': 'accessories',
    'brand name touch (4th generation) accessories': 'accessories',
    'brand name touch (4th generation) covers': 'accessories',
    'brand name (5th generation) accessories': 'accessories',
    'brand name (5th generation) covers': 'accessories',

    'books & magazines': 'media',
    'software & books': 'media',
    'book accessories': 'media',
    'movies': 'media',
    'music': 'media',

    'walmart for business': 'retail',
    'top rated': 'retail',
    'clearance': 'retail',
    'frys': 'retail',
    'categories': 'retail',
    'featured brands': 'retail',
}


In [52]:
def normalize_category_list(cat_list, norm_map):
    normalized = []
    for cat in cat_list:
        key = cat.strip().lower()
        if key in norm_map:
            mapped = norm_map[key]
            if mapped not in normalized:
                normalized.append(mapped)
        else:
            if key not in normalized:
                normalized.append(key)
    return normalized


In [53]:
data['category_list']

0        [electronics, ipad & tablets, tablets, compute...
1        [electronics, ipad & tablets, tablets, compute...
2        [electronics, ipad & tablets, tablets, compute...
3        [electronics, ipad & tablets, tablets, compute...
4        [electronics, ipad & tablets, tablets, compute...
                               ...                        
27895    [stereos, remote controls, audio docks & mini ...
27896    [stereos, remote controls, audio docks & mini ...
27897    [stereos, remote controls, audio docks & mini ...
27898    [stereos, remote controls, audio docks & mini ...
27899    [stereos, remote controls, audio docks & mini ...
Name: category_list, Length: 27898, dtype: object

In [54]:
data['normalized_categories'] = data['category_list'].apply(
    lambda x: normalize_category_list(x, normalization_map)
)

In [55]:
data['normalized_categories']

0              [electronics, tablets, computers & tablets]
1              [electronics, tablets, computers & tablets]
2              [electronics, tablets, computers & tablets]
3              [electronics, tablets, computers & tablets]
4              [electronics, tablets, computers & tablets]
                               ...                        
27895    [stereos, remote controls, audio, kitchen & di...
27896    [stereos, remote controls, audio, kitchen & di...
27897    [stereos, remote controls, audio, kitchen & di...
27898    [stereos, remote controls, audio, kitchen & di...
27899    [stereos, remote controls, audio, kitchen & di...
Name: normalized_categories, Length: 27898, dtype: object

In [56]:
data.drop(['categories', 'categories_clean', 'category_list'], axis=1, inplace=True)

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27898 entries, 0 to 27899
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   product                27898 non-null  object 
 1   doRecommend            27898 non-null  float64
 2   rating                 27898 non-null  float64
 3   reviews                27898 non-null  object 
 4   title                  27898 non-null  object 
 5   feedback               27898 non-null  object 
 6   normalized_categories  27898 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.7+ MB


Combine categories for same products

In [58]:
def merge_categories(series_of_lists):
    all_tags = set()
    for tags in series_of_lists:
        all_tags.update(tags)
    return list(all_tags)

In [59]:
# Step 1: Group by product, merge categories
merged = data.groupby('product').agg({
    'normalized_categories': merge_categories
}).reset_index()

# Step 2: Merge back to the original data on 'product'
data = data.drop('normalized_categories', axis=1).merge(merged, on='product', how='left')

# Sentiment Analysis of Feedback Column

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/shysu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/shysu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [64]:
stp_words=stopwords.words('english')
def clean_review(review):
  cleanreview=" ".join(word for word in review.
                       split() if word not in stp_words)
  return cleanreview

data['feedback']=data['feedback'].apply(clean_review)

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [66]:
cv = TfidfVectorizer(max_features=2500)
X = cv.fit_transform(data['feedback'] ).toarray()
y = data['rating'].apply(lambda x: 1 if x >= 4 else 0)

In [67]:
model = LogisticRegression()
model.fit(X, y)

# Predict sentiment
data['pred_sentiment'] = model.predict(X)

# Aggregations

In [68]:
summary = data.groupby('product').agg({
    'rating': ['mean', 'count'],
    'pred_sentiment': 'mean',
    'doRecommend': 'mean',
    'normalized_categories': 'first'
}).reset_index()

summary.columns = ['product', 'avg_rating', 'review_count', 'pos_sentiment_ratio', 'avg_recommend', 'categories']

summary['rank'] = summary['avg_rating']*summary['review_count']*summary['pos_sentiment_ratio']
summary = summary.sort_values('rank', ascending=False)

In [69]:
summary.head()

,product,avg_rating,review_count,pos_sentiment_ratio,avg_recommend,categories,rank
40,tablet 7 display wifi 8 gb magenta,4.453443,10965,0.967259,0.950570,"[computers, computers & tablets, electronics f...",47233.214045
35,paperwhite ebook reader 4 gb 6 monochrome p...,4.755038,3176,0.992128,0.979534,"[office electronics, computers & tablets, elec...",14983.124055
41,tablet a 101 tablet 8 hd display wifi 16 gb m...,4.586709,2814,0.979744,0.963042,"[tablets, computers & tablets, electronics]",12645.557569
53,tv_x000d_ tv,4.650040,2526,0.987332,0.959224,"[retail, electronics features, surveillance, w...",11597.198733
15,electonics brand home_x000d_\nelectonics brand...,4.693333,2100,0.989048,0.964286,"[retail, electronics features, surveillance, w...",9748.053333


In [70]:
def expand_rows(df):
    return df.explode('categories')

category_summary = expand_rows(summary)

top_per_category = category_summary.sort_values(['categories', 'rank'], ascending=False)
top_5 = top_per_category.groupby('categories').head(5)

In [ ]:
top_5

,product,avg_rating,review_count,pos_sentiment_ratio,avg_recommend,categories,rank
43,tv tv,4.650178,2527,0.987337,0.959240,wireless speakers,11602.194302
11,electonics brand home\nelectonics brand home,4.693333,2100,0.989048,0.964286,wireless speakers,9748.053333
10,electonics brand home,4.563531,1212,0.977723,0.954620,wireless speakers,5407.784653
37,tap portable bluetooth and wifi speaker black...,4.729560,318,0.993711,0.974843,wireless speakers,1494.540881
2,5w usb charger and power adapter for tablets a...,4.458937,207,0.971014,0.864734,wireless speakers,896.246377
...,...,...,...,...,...,...,...
6,brand ipad16gb 7 ips display tablet wifi 16 gb...,4.552987,1038,0.967245,0.961464,accessories,4571.198459
33,tablet 7 display wifi 8 gb black,4.604839,372,0.970430,0.951613,accessories,1662.346774
2,5w usb charger and power adapter for tablets a...,4.458937,207,0.971014,0.864734,accessories,896.246377
5,all pocketbook touch hd 3 ereader black 6 gla...,4.429245,212,0.948113,0.849057,accessories,890.278302


In [ ]:
top_5_grouped = top_5.groupby('product').agg({
    'categories': lambda x: list(set(x)),
    'avg_rating': 'first',
    'pos_sentiment_ratio': 'first',
    'avg_recommend': 'first',
    'review_count': 'first',
    'rank': 'first'
}).reset_index()

In [ ]:
top_5_grouped = top_5_grouped.sort_values('rank', ascending=False)

In [ ]:
top_5_grouped.shape

(21, 7)

In [ ]:
top_5_grouped.to_excel('output.xlsx', index=False)